# Draft Version Similarity Analysis

This notebook analyzes lexical changes between successive drafts of the Geothermiebeschleunigungsgesetz using TF-IDF cosine similarity.

The analysis tracks how the legislative text evolves from:
- v0 (03.07) - Initial ministerial draft
- v1 (15.08) - Revised government draft
- v2 (01.10) - Parliamentary introduction
- v3 (03.12) - Final committee report


## 1. Setup and Data Loading


In [1]:
import re
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import openpyxl
EXCEL_PATH = "../data/geothermie_gesetz_kommentare.xlsx"
SHEET_NAME = "Gesetz + Kommentare"

df = pd.read_excel(EXCEL_PATH, sheet_name=SHEET_NAME)
df.head()

,Artikel,Typ,Paragraph,Absatz,Gliederungspunkt_Nr,Gesetzestext_Entwurf_1_0307,Gesetzestext_Entwurf_2_1508,Gesetzestext_Entwurf_3_0110,Gesetzestext_Entwurf_4_0312,Org_2,...,Org_24,Org_25,Org_26,Org_27,Org_28,Org_29,Org_30,Org_31,Org_32,Org_33
0,1,Allgemeine Anmerkungen,-1,0,NaN,NaN,NaN,NaN,NaN,Der Beschleunigungseffekt des GeoBG erscheint ...,...,NaN,NaN,NaN,NaN,"Grundsätzlich ist ein Mehr an Geothermie, Sole...",Die Dekarbonisierung der Wärmeversorgung ist e...,s.o. § 1,NaN,NaN,NaN
1,1,Paragraph/Absatz,1,0,NaN,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Paragraph/Absatz,2,0,NaN,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Paragraph/Absatz,3,0,NaN,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Paragraph/Absatz,4,0,NaN,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Text Preprocessing


### Minimal Text Cleaning

In [3]:
# Pattern to match hyphenation across line breaks
HYPHEN_LINEBREAK_RE = re.compile(r"(\w+)-\s*\n\s*(\w+)", flags=re.UNICODE)

# Pattern to normalize slashed compounds
SLASH_JOIN_RE = re.compile(r"(\w+)\s*/\s*(\w+)", flags=re.UNICODE)


def clean_legal_text(text: Optional[str]) -> str:
    """
    Cleans legal text while preserving semantic hyphenation.
    Only removes hyphens caused by line breaks (e.g. PDF artifacts).
    """
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return ""

    t = str(text)

    # Normalize line endings
    t = t.replace("\r\n", "\n").replace("\r", "\n")

    # Normalize tabs
    t = t.replace("\t", " ")

    # Remove hyphenation ONLY when caused by line breaks
    t = HYPHEN_LINEBREAK_RE.sub(r"\1\2", t)

    # Replace remaining newlines with spaces
    t = re.sub(r"\n+", " ", t)

    # Optional: normalize slashed compounds (use with care)
    # Example: "Wärme-/Kältespeicher" -> "Wärme- und Kältespeicher"
    t = SLASH_JOIN_RE.sub(r"\1 und \2", t)

    # Collapse whitespace
    t = re.sub(r"\s+", " ", t).strip()

    return t

In [4]:
VERSION_COLS = [
    'Gesetzestext_Entwurf_1_0307', # Referentenentwurf
    'Gesetzestext_Entwurf_2_1508', # 1. RegE (oder Zwischenstand)
    'Gesetzestext_Entwurf_3_0110', # 2. RegE / BT-Drucksache
    'Gesetzestext_Entwurf_4_0312' # Ausschussbericht / finaler Stand
]

# directly apply cleaning function to every column
df2 = df.copy()
for c in VERSION_COLS:
    df2[c + "_clean"] = df2[c].apply(clean_legal_text)

df_versions = df2[df2["Typ"] != "Allgemeine Anmerkungen"].copy()

df_versions = df_versions.rename(columns={
    "Gesetzestext_Entwurf_1_0307_clean": "Gesetzestext_v0_clean",
    "Gesetzestext_Entwurf_2_1508_clean": "Gesetzestext_v1_clean",
    "Gesetzestext_Entwurf_3_0110_clean": "Gesetzestext_v2_clean",
    "Gesetzestext_Entwurf_4_0312_clean": "Gesetzestext_v3_clean",
})


df_versions.head()

,Artikel,Typ,Paragraph,Absatz,Gliederungspunkt_Nr,Gesetzestext_Entwurf_1_0307,Gesetzestext_Entwurf_2_1508,Gesetzestext_Entwurf_3_0110,Gesetzestext_Entwurf_4_0312,Org_2,...,Org_28,Org_29,Org_30,Org_31,Org_32,Org_33,Gesetzestext_v0_clean,Gesetzestext_v1_clean,Gesetzestext_v2_clean,Gesetzestext_v3_clean
1,1,Paragraph/Absatz,1,0,NaN,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,Zweck und Ziel des Gesetzes\nZweck dieses Gese...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Zweck und Ziel des Gesetzes Zweck dieses Geset...,Zweck und Ziel des Gesetzes Zweck dieses Geset...,Zweck und Ziel des Gesetzes Zweck dieses Geset...,Zweck und Ziel des Gesetzes Zweck dieses Geset...
2,1,Paragraph/Absatz,2,0,NaN,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,Anwendungsbereich\nDieses Gesetz ist anzuwende...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Anwendungsbereich Dieses Gesetz ist anzuwenden...,Anwendungsbereich Dieses Gesetz ist anzuwenden...,Anwendungsbereich Dieses Gesetz ist anzuwenden...,Anwendungsbereich Dieses Gesetz ist anzuwenden...
3,1,Paragraph/Absatz,3,0,NaN,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,Begriffsbestimmungen\nIm Sinne dieses Gesetzes...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Begriffsbestimmungen Im Sinne dieses Gesetzes ...,Begriffsbestimmungen Im Sinne dieses Gesetzes ...,Begriffsbestimmungen Im Sinne dieses Gesetzes ...,Begriffsbestimmungen Im Sinne dieses Gesetzes ...
4,1,Paragraph/Absatz,4,0,NaN,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,Überragendes öffentliches Interesse\nDie Erric...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Überragendes öffentliches Interesse Die Errich...,Überragendes öffentliches Interesse Die Errich...,Überragendes öffentliches Interesse Die Errich...,Überragendes öffentliches Interesse Die Errich...
5,1,Paragraph/Absatz,5,0,NaN,Vorzeitiger Beginn\nFür eine Anlage nach § 2 N...,Vorzeitiger Beginn\nFür eine Anlage nach § 2 N...,Vorzeitiger Beginn\nFür eine Anlage nach § 2 N...,Vorzeitiger Beginn\nFür eine Anlage nach § 2 b...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Vorzeitiger Beginn Für eine Anlage nach § 2 Nu...,Vorzeitiger Beginn Für eine Anlage nach § 2 Nu...,Vorzeitiger Beginn Für eine Anlage nach § 2 Nu...,Vorzeitiger Beginn Für eine Anlage nach § 2 be...


In [5]:
# Optional: Export cleaned data for reuse
# df_new.to_excel("../exports/geobg_cleaned.xlsx", index=False)

## 3. Article-Level Analysis

Aggregate text by article for high-level comparison.


### Aggregate text by Artikel (per version)

In [6]:
def aggregate_by_artikel(
    df: pd.DataFrame,
    artikel_col: str,
    text_col: str,
    sort_cols: list[str] | None = None,
) -> pd.DataFrame:
    """
    Aggregate sub-paragraphs into article-level text.
    """
    d = df.copy()

    if sort_cols:
        sort_cols = [c for c in sort_cols if c in d.columns]
        d = d.sort_values([artikel_col] + sort_cols)

    agg = (
        d.groupby(artikel_col, dropna=False)
         .agg(
             n_subunits=(text_col, lambda x: int(x.notna().sum())),
             text=(text_col, lambda x: "\n".join(
                 [t for t in x if isinstance(t, str) and t.strip()]
             ))
         )
         .reset_index()
    )

    agg["text_len"] = agg["text"].str.len()
    return agg

In [7]:
v0_art = aggregate_by_artikel(df_versions, "Artikel", "Gesetzestext_v0_clean", ["Paragraph","Absatz"])
v1_art = aggregate_by_artikel(df_versions, "Artikel", "Gesetzestext_v1_clean", ["Paragraph","Absatz"])
v2_art = aggregate_by_artikel(df_versions, "Artikel", "Gesetzestext_v2_clean", ["Paragraph","Absatz"])
v3_art = aggregate_by_artikel(df_versions, "Artikel", "Gesetzestext_v3_clean", ["Paragraph","Absatz"])

#### Build article-level comparison tables (v0→v1, v0→v3)

In [8]:
def build_artikel_pairs(a0: pd.DataFrame, aX: pd.DataFrame, suffixX: str):
    out = a0.merge(
        aX,
        on="Artikel",
        how="outer",
        suffixes=("_v0", f"_{suffixX}"),
        indicator=True,
    )

    out["text_v0"] = out["text_v0"].fillna("")
    out[f"text_{suffixX}"] = out[f"text_{suffixX}"].fillna("")

    out["status"] = out["_merge"].map({
        "both": "matched",
        "left_only": "removed",
        "right_only": "added"
    })

    # delete rows where text is empty in both versions
    out = out[~((out["text_v0"] == "") & (out[f"text_{suffixX}"] == ""))]

    out = out.drop(columns="_merge")
    return out


pairs_v01 = build_artikel_pairs(v0_art, v1_art, "v1")
pairs_v03 = build_artikel_pairs(v0_art, v3_art, "v3")

In [9]:
pairs_v01

,Artikel,n_subunits_v0,text_v0,text_len_v0,n_subunits_v1,text_v1,text_len_v1,status
0,1,29,Zweck und Ziel des Gesetzes Zweck dieses Geset...,12590,29,Zweck und Ziel des Gesetzes Zweck dieses Geset...,13022,matched
1,2,1,Änderung des Gesetzes über die Umweltverträgli...,747,1,Änderung des Gesetzes über die Umweltverträgli...,723,matched
2,3,1,Änderung der Verwaltungsgerichtsordnung der Di...,793,1,Änderung der Verwaltungsgerichtsordnung Die Ve...,802,matched
3,4,9,Änderung des Bundesberggesetzes Das Bundesberg...,8857,9,Änderung des Bundesberggesetzes Das Bundesberg...,9565,matched
4,5,6,Änderung des Wasserhaushaltsgesetzes Das Wasse...,3161,6,Änderung des Wasserhaushaltsgesetzes Das Wasse...,3269,matched
5,6,1,Inkrafttreten Dieses Gesetz tritt vorbehaltlic...,404,1,Inkrafttreten Dieses Gesetz tritt vorbehaltlic...,404,matched


#### Compute similarity (TF–IDF baseline, article level)

In [10]:
def add_tfidf_similarity(pairs: pd.DataFrame, colX: str) -> pd.DataFrame:
    texts = pairs["text_v0"].tolist() + pairs[colX].tolist()

    vec = TfidfVectorizer(ngram_range=(1, 2), min_df=1, max_df=0.95)
    X = vec.fit_transform(texts)

    A = X[: len(pairs)]
    B = X[len(pairs):]

    pairs["sim_tfidf"] = np.diag(cosine_similarity(A, B))
    pairs["delta"] = 1 - pairs["sim_tfidf"]

    return pairs


pairs_v01 = add_tfidf_similarity(pairs_v01, "text_v1")
pairs_v03 = add_tfidf_similarity(pairs_v03, "text_v3")

In [11]:
def summarize(pairs: pd.DataFrame):
    m = pairs[pairs["status"] == "matched"]

    return {
        "N_articles": len(pairs),
        "median_sim": m["sim_tfidf"].median(),
        "p90_delta": m["delta"].quantile(0.90),
        "pct_sim_ge_0_98": (m["sim_tfidf"] >= 0.98).mean() * 100,
    }


summary_v01 = summarize(pairs_v01)
summary_v03 = summarize(pairs_v03)

summary_v01, summary_v03

({'N_articles': 6,
  'median_sim': np.float64(0.9433729220703124),
  'p90_delta': np.float64(0.15018144680649365),
  'pct_sim_ge_0_98': np.float64(16.666666666666664)},
 {'N_articles': 10,
  'median_sim': np.float64(0.8068220396100233),
  'p90_delta': np.float64(1.0),
  'pct_sim_ge_0_98': np.float64(0.0)})

## 4. Paragraph-Level Similarity

Compute TF-IDF cosine similarity at the paragraph level for fine-grained change detection.


### Similarity at paragraph level (TF-IDF cosine)

In [12]:
@dataclass
class SimilarityConfig:
    ngram_range: Tuple[int, int] = (1, 2)   # unigrams + bigrams
    min_df: int = 1
    max_df: float = 0.95
    use_char_ngrams: bool = True         # set True if you want more robust matching against small edits


def tfidf_cosine_pairwise(a: List[str], b: List[str], cfg: SimilarityConfig) -> np.ndarray:
    a = [x or "" for x in a]
    b = [x or "" for x in b]

    a_empty = np.array([len(x.strip()) == 0 for x in a])
    b_empty = np.array([len(x.strip()) == 0 for x in b])

    sims = np.full(len(a), np.nan, dtype=float)

    # empty vs non-empty → 0
    sims[a_empty ^ b_empty] = 0.0

    # compute only where both are non-empty
    idx = ~(a_empty | b_empty)
    if idx.any():
        a_sub = [a[i] for i in np.where(idx)[0]]
        b_sub = [b[i] for i in np.where(idx)[0]]

        if cfg.use_char_ngrams:
            vectorizer = TfidfVectorizer(
                analyzer="char_wb",
                ngram_range=(3, 5),
                min_df=cfg.min_df,
                max_df=cfg.max_df,
            )
        else:
            vectorizer = TfidfVectorizer(
                ngram_range=cfg.ngram_range,
                min_df=cfg.min_df,
                max_df=cfg.max_df,
            )

        X = vectorizer.fit_transform(a_sub + b_sub)
        Xa = X[: len(a_sub)]
        Xb = X[len(a_sub):]

        from sklearn.preprocessing import normalize
        Xa = normalize(Xa)
        Xb = normalize(Xb)

        sims_sub = (Xa.multiply(Xb)).sum(axis=1).A1
        sims[idx] = sims_sub

    return sims


cfg = SimilarityConfig(ngram_range=(1, 2), use_char_ngrams=False)

v0 = df_versions["Gesetzestext_v0_clean"].tolist()
v1 = df_versions["Gesetzestext_v1_clean"].tolist()
v2 = df_versions["Gesetzestext_v2_clean"].tolist()
v3 = df_versions["Gesetzestext_v3_clean"].tolist()

df_versions["sim_v0_v1"] = tfidf_cosine_pairwise(v0, v1, cfg)
df_versions["sim_v1_v2"] = tfidf_cosine_pairwise(v1, v2, cfg)
df_versions["sim_v2_v3"] = tfidf_cosine_pairwise(v2, v3, cfg)
df_versions["sim_v0_v3"] = tfidf_cosine_pairwise(v0, v3, cfg)

df_versions[["sim_v0_v1", "sim_v1_v2", "sim_v2_v3", "sim_v0_v3"]].describe()

,sim_v0_v1,sim_v1_v2,sim_v2_v3,sim_v0_v3
count,47.000000,47.000000,51.000000,51.000000
mean,0.858290,0.986833,0.883101,0.757440
std,0.193372,0.027404,0.269549,0.289436
min,0.000000,0.908740,0.000000,0.000000
25%,0.811692,0.998603,0.933839,0.658177
50%,0.931532,1.000000,1.000000,0.861098
75%,0.993773,1.000000,1.000000,0.937302
max,1.000000,1.000000,1.000000,1.000000


In [13]:
print("Cosine similarity between version pairs (TF-IDF):")
print("--"*30)
print(df_versions[["sim_v0_v1", "sim_v1_v2", "sim_v2_v3", "sim_v0_v3"]].describe())

Cosine simiarlity between version pairs (TF-IDF):
------------------------------------------------------------
       sim_v0_v1  sim_v1_v2  sim_v2_v3  sim_v0_v3
count  47.000000  47.000000  51.000000  51.000000
mean    0.858290   0.986833   0.883101   0.757440
std     0.193372   0.027404   0.269549   0.289436
min     0.000000   0.908740   0.000000   0.000000
25%     0.811692   0.998603   0.933839   0.658177
50%     0.931532   1.000000   1.000000   0.861098
75%     0.993773   1.000000   1.000000   0.937302
max     1.000000   1.000000   1.000000   1.000000


In [14]:
# Count rows where both versions are empty
((df_versions["Gesetzestext_v0_clean"].str.strip() == "") &
 (df_versions["Gesetzestext_v1_clean"].str.strip() == "")).mean()

# empty vs non-empty rows count
((df_versions["Gesetzestext_v0_clean"].str.strip() == "") ^
 (df_versions["Gesetzestext_v1_clean"].str.strip() == "")).mean()

# do we still see spurious 1.0 for empty vs non-empty?
mask = ((df_versions["Gesetzestext_v0_clean"].str.strip() == "") ^
        (df_versions["Gesetzestext_v1_clean"].str.strip() == ""))
df_versions.loc[mask, "sim_v0_v1"].describe()


count    1.0
mean     0.0
std      NaN
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
Name: sim_v0_v1, dtype: float64

## 5. Summary Statistics

Analyze distribution of similarity scores across draft versions.


### “Distribution of differences” baseline

In [15]:
THR = 0.95  # Similarity threshold: 0.98 is strict, 0.90 is looser

df = df_versions.copy()

transitions = [("v0", "v1"), ("v1", "v2"), ("v2", "v3"), ("v0", "v3")]

# compute deltas + changed flags (NaNs stay NaN)
for a, b in transitions:
    sim_col = f"sim_{a}_{b}"
    df[f"delta_{a}_{b}"] = 1.0 - df[sim_col]
    # keep NaN if sim is NaN (empty-empty)
    df[f"changed_{a}_{b}"] = np.where(df[sim_col].notna(), df[sim_col] < THR, np.nan)

def transition_stats(df: pd.DataFrame, a: str, b: str, thr: float) -> dict:
    sim_col = f"sim_{a}_{b}"
    delta_col = f"delta_{a}_{b}"

    valid = df[sim_col].notna()  # excludes empty-empty
    n_valid = int(valid.sum())

    # share changed among valid comparisons only
    share_changed = (df.loc[valid, sim_col] < thr).mean() if n_valid > 0 else np.nan

    return {
        "transition": f"{a}→{b}",
        "n_paragraphs_total": len(df),
        "n_compared_valid": n_valid,
        "share_valid": n_valid / len(df) if len(df) else np.nan,
        "share_changed(<thr)_valid": share_changed,
        "median_delta_valid": df.loc[valid, delta_col].median() if n_valid > 0 else np.nan,
        "p90_delta_valid": df.loc[valid, delta_col].quantile(0.90) if n_valid > 0 else np.nan,
    }

summary = pd.DataFrame([transition_stats(df, a, b, THR) for a, b in transitions])

In [16]:
print("Summary of version-to-version changes (per paragraph):")
print("--"*30)
print(summary)

Summary of version-to-version changes (per paragraph):
------------------------------------------------------------
  transition  n_paragraphs_total  n_compared_valid  share_valid  \
0      v0→v1                  51                47     0.921569   
1      v1→v2                  51                47     0.921569   
2      v2→v3                  51                51     1.000000   
3      v0→v3                  51                51     1.000000   

   share_changed(<thr)_valid  median_delta_valid  p90_delta_valid  
0                   0.595745        6.846767e-02         0.409985  
1                   0.148936        1.110223e-16         0.063465  
2                   0.333333        7.216450e-15         0.267106  
3                   0.784314        1.389021e-01         0.542353  


In [17]:
for a,b in [("v0","v1"),("v1","v2"),("v2","v3"),("v0","v3")]:
    sim = df[f"sim_{a}_{b}"]
    valid = sim.notna()
    print(a,b,
          "share>=0.98:", (sim[valid] >= 0.98).mean(),
          "share==1.0:", (sim[valid] == 1.0).mean(),
          "share==0.0:", (sim[valid] == 0.0).mean())

v0 v1 share>=0.98: 0.3191489361702128 share==1.0: 0.0851063829787234 share==0.0: 0.02127659574468085
v1 v2 share>=0.98: 0.8085106382978723 share==1.0: 0.1276595744680851 share==0.0: 0.0
v2 v3 share>=0.98: 0.5490196078431373 share==1.0: 0.19607843137254902 share==0.0: 0.0784313725490196
v0 v3 share>=0.98: 0.1568627450980392 share==1.0: 0.0784313725490196 share==0.0: 0.09803921568627451


## 6. Detailed Change Analysis

Identify specific paragraphs with substantial changes for qualitative inspection.


### 4.2 Top changed paragraphs (for qualitative inspection)

In [18]:
KEY_COLS = [c for c in ["Artikel", "Paragraph", "Absatz", "Gliederungspunkt_Nr"] if c in df.columns]

top = (
    df.sort_values("delta_v0_v3", ascending=False)
      .loc[:, KEY_COLS + ["delta_v0_v3", "sim_v0_v3", "Gesetzestext_v0_clean", "Gesetzestext_v3_clean"]]
      .head(15)
)

top

,Artikel,Paragraph,Absatz,Gliederungspunkt_Nr,delta_v0_v3,sim_v0_v3,Gesetzestext_v0_clean,Gesetzestext_v3_clean
53,10,0,0,0.0,1.000000,0.000000,,Änderung des Wärmeplanungsgesetzes Das Wärmepl...
52,9,0,0,0.0,1.000000,0.000000,,Änderung des Baugesetzbuches Das Baugesetzbuch...
51,8,0,0,0.0,1.000000,0.000000,,Änderung der Verordnung zur Anrechnung von str...
34,4,1,0,1.0,1.000000,0.000000,,1. In der Inhaltsübersicht wird die Angabe zu ...
50,7,0,0,0.0,1.000000,0.000000,,Änderung des Bundes-Immissionsschutzgesetzes D...
39,4,6,0,6.0,0.542353,0.457647,5. § 56 wird wie folgt geändert: a) Nach Absat...,6. § 56 wird wie folgt geändert: a) Nach § 56 ...
12,1,8,0,NaN,0.457859,0.542141,"Planfeststellung, Plangenehmigung","Planfeststellung, Plangenehmigung, Enteignungs..."
37,4,4,0,4.0,0.443540,0.556460,3. In § 51 Absatz 3 werden nach den Angaben „v...,4. In § 51 Absatz 3 Satz 1 wird nach der Angab...
36,4,3,0,3.0,0.411270,0.588730,2. § 15 wird wie folgt geändert: a) Die Angabe...,3. § 15 wird durch den folgenden § 15 ersetzt:...
10,1,7,1,NaN,0.409037,0.590963,(1) Eigentümer und sonstige Nutzungsberechtigt...,(1) Eigentümer und sonstige Nutzungsberechtigt...


In [19]:
extra_v3 = df[
    (df["Gesetzestext_v3_clean"].str.len() > 0)
    & (df["Gesetzestext_v0_clean"].str.len() == 0)
]

extra_v3[KEY_COLS + ["Gesetzestext_v3_clean"]].head(20)

,Artikel,Paragraph,Absatz,Gliederungspunkt_Nr,Gesetzestext_v3_clean
34,4,1,0,1.0,1. In der Inhaltsübersicht wird die Angabe zu ...
50,7,0,0,0.0,Änderung des Bundes-Immissionsschutzgesetzes D...
51,8,0,0,0.0,Änderung der Verordnung zur Anrechnung von str...
52,9,0,0,0.0,Änderung des Baugesetzbuches Das Baugesetzbuch...
53,10,0,0,0.0,Änderung des Wärmeplanungsgesetzes Das Wärmepl...


In [20]:
mask_aligned = df["Gesetzestext_v0_clean"].str.len() > 0
df_aligned = df[mask_aligned].copy()

df_aligned["sim_v0_v3"].describe()

count    46.000000
mean      0.839770
std       0.149688
min       0.457647
25%       0.764930
50%       0.884552
75%       0.947085
max       1.000000
Name: sim_v0_v3, dtype: float64

In [21]:
sim = df_aligned["sim_v0_v3"]

summary_v0_v3 = {
    "mean": sim.mean(),
    "median": sim.median(),
    "p90": sim.quantile(0.90),
    "share_above_0.98": (sim > 0.98).mean(),
}

summary_v0_v3


{'mean': np.float64(0.8397704124431796),
 'median': np.float64(0.8845516321292525),
 'p90': np.float64(1.0),
 'share_above_0.98': np.float64(0.17391304347826086)}

In [22]:
delta_v0_v3 = 1 - df_aligned["sim_v0_v3"]
delta_v0_v3.describe()

count    4.600000e+01
mean     1.602296e-01
std      1.496876e-01
min     -2.220446e-16
25%      5.291526e-02
50%      1.154484e-01
75%      2.350699e-01
max      5.423535e-01
Name: sim_v0_v3, dtype: float64

In [23]:
THR = 0.98

changed = df_aligned[df_aligned["sim_v0_v3"] < THR]
unchanged = df_aligned[df_aligned["sim_v0_v3"] >= THR]

{
    "pct_changed": len(changed) / len(df_aligned),
    "median_changed": changed["sim_v0_v3"].median(),
    "median_unchanged": unchanged["sim_v0_v3"].median(),
}


{'pct_changed': 0.8260869565217391,
 'median_changed': np.float64(0.8546142208041305),
 'median_unchanged': np.float64(1.0)}